In [ ]:
!pip install transformers datasets torch huggingface-hub


In [ ]:
from huggingface_hub import notebook_login

notebook_login()

#from huggingface_hub import HfApi
#api = HfApi()

In [ ]:
from datasets import load_dataset
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification
from transformers import Trainer, TrainingArguments

# Step 1: Load the dataset
dataset = load_dataset("imdb")

# Step 2: Preprocess the dataset
tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

def tokenize_data(example):
    return tokenizer(example['text'], padding="max_length", truncation=True, max_length=256)

tokenized_dataset = dataset.map(tokenize_data, batched=True)

# Remove unnecessary columns
tokenized_dataset = tokenized_dataset.remove_columns(["text"])
tokenized_dataset = tokenized_dataset.rename_column("label", "labels")
tokenized_dataset.set_format("torch")

# Step 3: Load the model
model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)

# Step 4: Define training arguments
training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    logging_steps=100,
    save_steps=500,
)

# Step 5: Define the Trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
)

# Step 6: Train the model
trainer.train()

# Step 7: Save the fine-tuned model
model.save_pretrained("./sentiment-analysis-model")
tokenizer.save_pretrained("./sentiment-analysis-model")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md:   0%|          | 0.00/7.81k [00:00<?, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/21.0M [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/20.5M [00:00<?, ?B/s]

unsupervised-00000-of-00001.parquet:   0%|          | 0.00/42.0M [00:00<?, ?B/s]

...

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


Epoch,Training Loss,Validation Loss
1,0.250600,0.257972
2,0.157000,0.229598
3,0.111500,0.209263


('./sentiment-analysis-model/tokenizer_config.json',
 './sentiment-analysis-model/special_tokens_map.json',
 './sentiment-analysis-model/vocab.txt',
 './sentiment-analysis-model/added_tokens.json')

In [ ]:
from transformers import pipeline, DistilBertTokenizer, DistilBertForSequenceClassification

# Define test samples
test_samples = [
    "This movie was absolutely fantastic! The plot, acting, and visuals were top-notch.",
    "the movie has great comedy element but it lacks good stories plot",
    "An average movie with some good moments but mostly forgettable.",
]

# Step 1: Load the base model
base_model = DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=2)
base_tokenizer = DistilBertTokenizer.from_pretrained("distilbert-base-uncased")

base_pipeline = pipeline("text-classification", model=base_model, tokenizer=base_tokenizer)

# Step 2: Load the fine-tuned model
fine_tuned_model = DistilBertForSequenceClassification.from_pretrained("rautaditya/sentiment-analysis-model-imdb")
fine_tuned_tokenizer = DistilBertTokenizer.from_pretrained("rautaditya/sentiment-analysis-model-imdb")

fine_tuned_pipeline = pipeline("text-classification", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer)

# Step 3: Test both models
print("Base Model Predictions:")
for text in test_samples:
    result = base_pipeline(text)
    print(f"Text: {text}")
    #print(f"Prediction: {result}\n")
    for res in result:
      label_text = "positive" if res['label'] == 'LABEL_1' else "negative"
      print(f"Prediction: {label_text} with score: {res['score']:.2f}")


print("Fine-Tuned Model Predictions:")
for text in test_samples:
    result = fine_tuned_pipeline(text)
    print(f"Text: {text}")
    #print(f"Prediction: {result}\n")
    for res in result:
      label_text = "positive" if res['label'] == 'LABEL_1' else "negative"
      print(f"Prediction: {label_text} with score: {res['score']:.2f}")



/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

...

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.25k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/695 [00:00<?, ?B/s]

Base Model Predictions:
Text: This movie was absolutely fantastic! The plot, acting, and visuals were top-notch.
Prediction: positive with score: 0.55
Text: the movie has great comedy element but it lacks good stories plot
Prediction: positive with score: 0.55
Text: An average movie with some good moments but mostly forgettable.
Prediction: positive with score: 0.54
Fine-Tuned Model Predictions:
Text: This movie was absolutely fantastic! The plot, acting, and visuals were top-notch.
Prediction: positive with score: 1.00
Text: the movie has great comedy element but it lacks good stories plot
Prediction: negative with score: 0.97
Text: An average movie with some good moments but mostly forgettable.
Prediction: negative with score: 0.99


In [ ]:

repo_name = "rautaditya/sentiment-analysis-model-imdb"  # Replace with your Hugging Face username and desired repository name
fine_tuned_tokenizer.push_to_hub(repo_name)
fine_tuned_model.push_to_hub(repo_name)

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/rautaditya/sentiment-analysis-model-imdb/commit/08c495393c8d1bacbd44b471700d1fc3e4f8f3d5', commit_message='Upload DistilBertForSequenceClassification', commit_description='', oid='08c495393c8d1bacbd44b471700d1fc3e4f8f3d5', pr_url=None, repo_url=RepoUrl('https://huggingface.co/rautaditya/sentiment-analysis-model-imdb', endpoint='https://huggingface.co', repo_type='model', repo_id='rautaditya/sentiment-analysis-model-imdb'), pr_revision=None, pr_num=None)

In [ ]:
fine_tuned_tokenizer.push_to_hub(repo_name)
model.push_to_hub(repo_name)

In [ ]:
test_samples = [
    "This movie was absolutely fantastic! The plot, acting, and visuals were top-notch.",
    "the movie have a deep cultural roots but its boring",
    "An average movie with some good moments but mostly forgettable.",
]
# Step 3: Test both models
print("Base Model Predictions:")
for text in test_samples:
    result = base_pipeline(text)
    print(f"Text: {text}")
    #print(f"Prediction: {result}\n")
    for res in result:
      label_text = "positive" if res['label'] == 'LABEL_1' else "negative"
      print(f"Prediction: {label_text} with score: {res['score']:.2f}")


print("Fine-Tuned Model Predictions:")
for text in test_samples:
    result = fine_tuned_pipeline(text)
    print(f"Text: {text}")
    #print(f"Prediction: {result}\n")
    for res in result:
      label_text = "positive" if res['label'] == 'LABEL_1' else "negative"
      print(f"Prediction: {label_text} with score: {res['score']:.2f}")

Base Model Predictions:
Text: This movie was absolutely fantastic! The plot, acting, and visuals were top-notch.
Prediction: positive with score: 0.52
Text: the movie have a deep cultural roots but its boring
Prediction: positive with score: 0.52
Text: An average movie with some good moments but mostly forgettable.
Prediction: positive with score: 0.53
Fine-Tuned Model Predictions:
Text: This movie was absolutely fantastic! The plot, acting, and visuals were top-notch.
Prediction: positive with score: 1.00
Text: the movie have a deep cultural roots but its boring
Prediction: negative with score: 1.00
Text: An average movie with some good moments but mostly forgettable.
Prediction: negative with score: 0.99


In [ ]:
    "This movie was absolutely fantastic! The plot, acting, and visuals were top-notch.",
    "the movie have a deep cultural roots but its boring",
    "An average movie with some good moments but mostly forgettable.",

In [ ]:
test_samples = [
    "This movie was absolutely fantastic! The plot, acting, and visuals were top-notch.",
    "the movie have a deep cultural roots but its boring",
    "An average movie with some good moments but mostly forgettable.",
]